In [1]:
pwd

'/home/rnshishir/deepmed/CPRD_Cut22/debug'

In [2]:
import pyspark

In [3]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.functions import merge
from utils.utils import save_obj
from CPRD.functions.MedicalDictionary import * 
from CPRD.functions.Prediction import * 
from CPRD.functions.cohort_select_causal import * 

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [5]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.txt',
  'op_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_09

# cohort selection
### Association to study: effect of antihypertensives on ischaemic conditions

-Cohort selection: age between 60 and 61 years in years between 2009 and 2010

-take the first initiation of antihyp in this time to be baseline

-take random baseline for those without any antihyp

In [7]:
# medical dict can give us both exposures and outcomes codes - e.g. diabetes as outcomes or antihyyp as exposurea
md = MedicalDictionaryRiskPrediction(file, spark)
temp = md.queryMedication(md.findItem('antihy'), merge= True)['merged']
expcodes = {}
expcodes['prodcode'] = temp['prodcode']

exposure selection

In [18]:
# CohortSoftCut from the causal cohort selection package has everything to select cohort and baseline
# specifically the baseline for those WITH the exposure is date of exposure, and for those WITHOUT exp of interest is random sampling of baseline 
cohortSelector = CohortSoftCut(1,60, 61,  expcodes, False, False, True)

In [19]:
# no mapping as you don't want to drop the prodcodes which are not mapped...
# medications = retrieve_medications(file, spark, mapping='none', duration=(2009, 2010), demographics=cohort, practiceLink=True)
# medications.write.parquet('/home/shared/shishir/AurumOut/rawDat/meds_nomapping_2009_2010_association_example.parquet')

medications = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/meds_nomapping_2009_2010_association_example.parquet')
# medications.select('patid').dropDuplicates().count()  - 208603 patients have meds in the time period
# medications.select('patid').count() - 11764962 number of records

# pipeline() function has 3 components:
1) demo extract gets eligible patients between age 60 and 61 ^ defined above and years 2009 and 2010  
2) extraction of the exposure of interest -  set  baseline as the date of the exposure  
3) for those without exposure (i.e. control patients) , set up baseline as randomised baseline


In [13]:
cohort = cohortSelector.pipeline(file, spark, duration=('2009-01-01', '2010-01-01'), randomNeg=True, sourceT=medications, sourceCol='prodcode', rollingTW=-1)
cohort.write.parquet('/home/shared/shishir/AurumOut/rawDat/cohort_association_example.parquet')


In [17]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/cohort_association_example.parquet')
cohort.count()
# 259345 pats 

225564

outcome selection

In [16]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/cohort_association_example.parquet')

necessaryColumns = ['patid',
 'gender',
 'dob',
 'study_entry',
 'startdate',
 'enddate',
  'exp_label']

cohort = cohort.select(necessaryColumnsaryColumns)

In [17]:
# label codes phenotyping from medical dict - ie maybe ischaemic conditions
labelcodes  = md.queryDisease(md.findItem('ischaem'), merge= True)['merged']
allIschaemiaCodes = labelcodes['medcode'] + labelcodes['ICD10'] +labelcodes['OPCS']

In [18]:
# label codes phenotyping from medical dict - ie maybe ischaemic conditions

# split diags into icd and nonicd(medcode) and re-union as "code"
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
GPdiags = allDiag[allDiag.source=='CPRD']
GPdiags = GPdiags.select(['patid' , 'eventdate', 'medcode']).withColumnRenamed('medcode', 'code')
HESdiags = allDiag[allDiag.source=='HES']
HESdiags = HESdiags.select(['patid' , 'eventdate', 'ICD']).withColumnRenamed('ICD', 'code')
allDiag = GPdiags.union(HESdiags)

# read death registry as death is an important data source for looking for outcome  
death = tables.retrieve_death(dir=file['death'], spark=spark)

In [20]:
# now we use the risk prediction label capture class
# basically with baseline we can capture 1) outcome, 2) time to outcome

# the exclusion_codes is those we should exclude based on condition - i.e., exclude those with prior cancers 
# the duration is time we should consider the records in the outcome space (maybe from 2008 since earliest baseline is 2009 and end is 2020)
# the follow_up_duration_month is number of months for the followup
# the time_to_event_mark_default is mark as -1 if no event and lasts till end of follow-up
# more information in package
risk_pred_generator = OutcomePrediction(label_condition=allIschaemiaCodes, 
                                             exclusion_codes=None, 
                                             duration=(2008, 2020), 
                                             follow_up_duration_month=24,
                                             time_to_event_mark_default=-1)



In [21]:

# demographics is the cohort file with sutdy entry etc
# source is the diag table
# source_col is column that has the diags
# exclusion_source is True if we want to exclude based on past diags

# check_death is true if we are to check death 
# column_condition is column that has the diags or meds or whatever modality we are wanting to look for label
# incidence is True if we are looking for incident lable
# prevalent_conditions is if incidence is false, then what are some prevalent conditions we are allowing to look for (a subset of the labels)
# more information in package



risk_cohort = risk_pred_generator.pipeline( demographics=cohort, 
                                             source=allDiag, 
                                              exclusion_source=False,
                                             check_death=True, 
                                             death=death,
                                             column_condition='code', 
                                             incidence=True, 
                                             prevalent_conditions=None)



risk_cohort. write.parquet('test.parquet')